In [1]:
pip install -U groq

In [1]:
from groq import Groq
client = Groq(api_key='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')

In [3]:
class Agent:
    def __init__(self, client: Groq, system: str = "") -> None:
        self.client = client
        self.system = system
        self.messages: list = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message=""):
        if message:
            self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            model="llama3-70b-8192", messages=self.messages
        )
        return completion.choices[0].message.content


In [4]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

FetchBitcoinPrice:
e.g.FetchBitcoinPrice: crypto symbol like BTCUSDT (cryptocurrency trading pair that represents the exchange rate between Bitcoin (BTC) and Tether (USDT))
Using the given API get the current BTCUSDT price

LanguageTranslation:
e.g.LanguageTranslation: input query to Translate to english
Returns an english query after translating it from the source query language

Example session 1:

Question: What is the current bitcoin price
Thought: I should look up FetchBitcoinPrice Function
Action: FetchBitcoinPrice: BTCUSDT
PAUSE

You will be called again with this:

Observation: The current bitcoin price is 6234925
Thought: I think I have found the answer
Action: 6234925.
You should then call the appropriate action and determine the answer from the result

You then output:

Answer: The current bitcoin price is 6234925

Example session 2:

Question: Wie ist der aktuelle Bitcoin-Preis?
Thought: This is not English language, so I need to translate first this to english
Action: LanguageTranslation : "Wie ist der aktuelle Bitcoin-Preis?"
PAUSE

You will be called again with this:

Observation: What is the current bitcoin price

Thought: Now i got the original query in english language,Now I should look up FetchBitcoinPrice Function
Action: FetchBitcoinPrice : BTCUSDT
PAUSE

You will be called again with this:

Observation: The current bitcoin price is 8934712

If you have the answer, output it as the Answer.

Answer: The current bitcoin price is 8934712

Now it's your turn:
""".strip()

In [5]:
import re
import requests
def FetchBitcoinPrice(symbol: str) -> int:
    """Fetches the latest Bitcoin price from Binance API with error handling."""
    try:
        print("inside FetchBitcoinPrice")
        url = f"https://api.binance.com/api/v3/ticker/price?symbol={symbol}"
        response = requests.get(url, timeout=5)  # Set a timeout for request
        
        response.raise_for_status()  # Raises an HTTPError for bad responses (4xx, 5xx)
        data = response.json()
        
        if "price" not in data:
            raise KeyError("Missing 'price' in response data")
        
        return float(data["price"])  # Convert to float for better handling
    except requests.exceptions.RequestException as e:
        print(f"Network error: {e}")
    except ValueError:
        print("Invalid response format received")
    except KeyError as e:
        print(f"Invalid response structure: {e}")
    
    return -1  # Return a default error value

def LanguageTranslation(sentence: str) -> str:
    """Translates a given German sentence to English with error handling."""
    try:
        if not isinstance(sentence, str):
            raise TypeError("Input must be a string")
        
        translations = {
            "Wie hoch ist der aktuelle Bitcoin-Preis?": "What is the current price of Bitcoin?",
            "Wie viel kostet Bitcoin gerade?": "How much does Bitcoin cost right now?",
            "Wie ist der neueste Bitcoin-Wert?": "What’s the latest Bitcoin value?",
            "Kannst du mir den Bitcoin-Kurs sagen?": "Can you tell me the Bitcoin rate?",
            "Zu welchem Preis wird Bitcoin heute gehandelt?": "What is Bitcoin trading at today?",
            "Wie viel ist ein Bitcoin derzeit wert?": "How much is one Bitcoin worth currently?",
            "Was ist der aktuelle Marktpreis von Bitcoin": "How much is one Bitcoin worth currently?",
            "Wie hoch ist der derzeitige Wert von Bitcoin?": "What is the present value of Bitcoin?",
            "Für wie viel wird Bitcoin heute verkauft?": "How much is Bitcoin selling for today?",
            "Wie lautet der aktuelle Bitcoin-Wechselkurs?": "What’s the latest Bitcoin exchange rate?"
        }
        
        return translations.get(sentence, "Translation not found")
    
    except TypeError as e:
        return f"Error: {e}"
    except Exception as e:
        return f"An unexpected error occurred: {e}"

In [6]:
def loop(max_iterations=10, query: str = ""):
    agent = Agent(client=client, system=system_prompt)
    tools = ["FetchBitcoinPrice", "LanguageTranslation"]
    next_prompt = query
    i = 0
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        if "PAUSE" in result and "Action" in result:
            pattern = r'Action:\s*([a-zA-Z_]+)\s*:\s*"?([^"\n]+)"?'
            action = re.findall(pattern, result, re.IGNORECASE)
            #action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            print("ACTION IS")
            print(action)
            chosen_tool = action[0][0]
            arg = action[0][1]
            if chosen_tool in tools:
                result_tool = eval(f"{chosen_tool}('{arg}')")
                next_prompt = f"Observation: {result_tool}"
            else:
                next_prompt = "Observation: Tool not found"
            print("next_prompt IS")
            print(next_prompt)
            continue
        if "Answer" in result:
            break


In [7]:
loop(query="Wie hoch ist der aktuelle Bitcoin-Preis")

AuthenticationError: Error code: 401 - {'error': {'message': 'Invalid API Key', 'type': 'invalid_request_error', 'code': 'invalid_api_key'}}

In [7]:
loop(query="create a paragraph about current bitcoin price ")

self.messages now
[{'role': 'system', 'content': 'You run in a loop of Thought, Action, PAUSE, Observation.\nAt the end of the loop you output an Answer\nUse Thought to describe your thoughts about the question you have been asked.\nUse Action to run one of the actions available to you - then return PAUSE.\nObservation will be the result of running those actions.\n\nYour available actions are:\n\nFetchBitcoinPrice:\ne.g.FetchBitcoinPrice: crypto symbol like BTCUSDT (cryptocurrency trading pair that represents the exchange rate between Bitcoin (BTC) and Tether (USDT))\nUsing the given API get the current BTCUSDT price\n\nLanguageTranslation:\ne.g.LanguageTranslation: input query to Translate to english\nReturns an english query after translating it from the source query language\n\nExample session 1:\n\nQuestion: What is the current bitcoin price\nThought: I should look up FetchBitcoinPrice Function\nAction: FetchBitcoinPrice: BTCUSDT\nPAUSE\n\nYou will be called again with this:\n\nObs